# BERT Implementation

In [3]:
from flair.data import Sentence
from flair.models import TextClassifier

classifier = TextClassifier.load('en-sentiment') #English-sentiment model pre-trained with IMDB movie reviews.

2019-11-09 19:56:10,297 loading file /home/prw0007/.flair/models/imdb.pt


/share/apps/rc/software/Anaconda3/5.3.1/envs/DeepNLP/lib/python3.6/site-packages/torch/serialization.py:573: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


In [4]:
sentence = Sentence('This film hurts. It is so bad that I am confused.')

# predict NER tags
classifier.predict(sentence)

# print sentence with predicted labels
print(sentence.labels)

[NEGATIVE (0.9706420302391052)]


# Word Embeddings - test

In [4]:
from flair.embeddings import WordEmbeddings

# init embedding
glove_embedding = WordEmbeddings('glove')

In [5]:
# create sentence.
sentence = Sentence('The grass is green .')

# embed a sentence using glove.
glove_embedding.embed(sentence)

# now check out the embedded tokens.
for token in sentence:
    print(token)
    print(token.embedding)

Token: 1 The
tensor([-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344,
        -0.5755,  0.0875,  0.2879, -0.0673,  0.3091, -0.2638, -0.1323, -0.2076,
         0.3340, -0.3385, -0.3174, -0.4834,  0.1464, -0.3730,  0.3458,  0.0520,
         0.4495, -0.4697,  0.0263, -0.5415, -0.1552, -0.1411, -0.0397,  0.2828,
         0.1439,  0.2346, -0.3102,  0.0862,  0.2040,  0.5262,  0.1716, -0.0824,
        -0.7179, -0.4153,  0.2033, -0.1276,  0.4137,  0.5519,  0.5791, -0.3348,
        -0.3656, -0.5486, -0.0629,  0.2658,  0.3020,  0.9977, -0.8048, -3.0243,
         0.0125, -0.3694,  2.2167,  0.7220, -0.2498,  0.9214,  0.0345,  0.4674,
         1.1079, -0.1936, -0.0746,  0.2335, -0.0521, -0.2204,  0.0572, -0.1581,
        -0.3080, -0.4162,  0.3797,  0.1501, -0.5321, -0.2055, -1.2526,  0.0716,
         0.7056,  0.4974, -0.4206,  0.2615, -1.5380, -0.3022, -0.0734, -0.2831,
         0.3710, -0.2522,  0.0162, -0.0171, -0.3898,  0.8742, -0.7257, -0.5106,
        -0.5203, -0.1459,  

# Create Corpus

In [8]:
import flair.datasets
corpus = flair.datasets.UD_ENGLISH()

# print the number of Sentences in the train split
print(len(corpus.train))

# print the number of Sentences in the test split
print(len(corpus.test))

# print the number of Sentences in the dev split
print(len(corpus.dev))


# print the first Sentence in the training split
print(corpus.test[0])

2019-11-09 19:19:25,633 Reading data from /home/prw0007/.flair/datasets/ud_english
2019-11-09 19:19:25,634 Train: /home/prw0007/.flair/datasets/ud_english/en_ewt-ud-train.conllu
2019-11-09 19:19:25,634 Test: /home/prw0007/.flair/datasets/ud_english/en_ewt-ud-test.conllu
2019-11-09 19:19:25,635 Dev: /home/prw0007/.flair/datasets/ud_english/en_ewt-ud-dev.conllu
12543
2077
2002
Sentence: "What if Google Morphed Into GoogleOS ?" - 7 Tokens


In [9]:
stats = corpus.obtain_statistics()
print(stats)

{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 12543,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 204585,
            "min": 1,
            "max": 159,
            "avg": 16.310691222195647
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 2077,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 25096,
            "min": 1,
            "max": 81,
            "avg": 12.082811747713048
        }
    },
    "DEV": {
        "dataset": "DEV",
        "total_number_of_documents": 2002,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 25148,
            "min": 1,
            "max": 75,
            "avg": 12.561438561438562
        }
    }
}


In [ ]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

##Note: You will need to save your split CSV data files in the data_folder path with each file titled appropriately 
#       i.e. train.csv test.csv dev.csv. This is because the corpus initializers will automatically search for the 
#       train, dev, test splits in a folder.

# this is the folder in which train, test and dev files reside
data_folder = '/path/to/data'

# column format indicating which columns hold the text and label(s)
column_name_map = {4: "text", 1: "label_topic", 2: "label_subtopic"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter='\t',    # tab-separated files
) 

# Training a Model

In [2]:
from flair.data import Corpus
from flair.datasets import IMDB
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

# 1. get the corpus
corpus: Corpus = IMDB().downsample(0.1)
print(corpus)

2019-11-09 19:55:33,428 Reading data from /home/prw0007/.flair/datasets/imdb
2019-11-09 19:55:33,430 Train: /home/prw0007/.flair/datasets/imdb/train.txt
2019-11-09 19:55:33,432 Dev: None
2019-11-09 19:55:33,434 Test: None


AttributeError: 'NoneType' object has no attribute 'exists'

In [16]:
# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

In [17]:
# 3. make a list of word embeddings
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                   # FlairEmbeddings('news-forward'),
                   # FlairEmbeddings('news-backward'),
                   ]

[b'<unk>', b'O', b'S-person', b'S-corporation', b'B-product', b'I-product', b'E-product', b'S-location', b'B-location', b'E-location', b'B-person', b'E-person', b'S-product', b'S-group', b'I-person', b'B-corporation', b'E-corporation', b'B-creative-work', b'I-creative-work', b'E-creative-work', b'B-group', b'E-group', b'I-location', b'I-group', b'S-creative-work', b'<START>', b'<STOP>']


In [18]:
# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

/share/apps/rc/software/Anaconda3/5.3.1/envs/DeepNLP/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [19]:
# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [20]:
# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

In [23]:
# 7. start the training
trainer.train('resources/taggers/ag_news',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=10)

2019-11-09 19:42:30,042 ----------------------------------------------------------------------------------------------------
2019-11-09 19:42:30,043 Evaluation method: MICRO_F1_SCORE
2019-11-09 19:42:30,435 ----------------------------------------------------------------------------------------------------
2019-11-09 19:42:31,020 epoch 1 - iter 0/11 - loss 4.25442219
2019-11-09 19:42:31,449 epoch 1 - iter 1/11 - loss 3.96355748
2019-11-09 19:42:31,886 epoch 1 - iter 2/11 - loss 4.22805897
2019-11-09 19:42:32,425 epoch 1 - iter 3/11 - loss 4.35936630
2019-11-09 19:42:32,828 epoch 1 - iter 4/11 - loss 4.79208441
2019-11-09 19:42:33,334 epoch 1 - iter 5/11 - loss 4.68821247
2019-11-09 19:42:33,824 epoch 1 - iter 6/11 - loss 5.01159436
2019-11-09 19:42:34,277 epoch 1 - iter 7/11 - loss 4.82517886
2019-11-09 19:42:34,801 epoch 1 - iter 8/11 - loss 4.98456658
2019-11-09 19:42:35,262 epoch 1 - iter 9/11 - loss 4.96149368
2019-11-09 19:42:35,655 epoch 1 - iter 10/11 - loss 4.89703738
2019-11-0

2019-11-09 19:43:54,068 ----------------------------------------------------------------------------------------------------
2019-11-09 19:43:54,690 epoch 8 - iter 0/11 - loss 1.49203074
2019-11-09 19:43:55,149 epoch 8 - iter 1/11 - loss 4.00940067
2019-11-09 19:43:55,582 epoch 8 - iter 2/11 - loss 3.73690037
2019-11-09 19:43:56,103 epoch 8 - iter 3/11 - loss 4.31594482
2019-11-09 19:43:56,618 epoch 8 - iter 4/11 - loss 4.12532446
2019-11-09 19:43:57,059 epoch 8 - iter 5/11 - loss 4.31468823
2019-11-09 19:43:57,454 epoch 8 - iter 6/11 - loss 4.31709325
2019-11-09 19:43:57,863 epoch 8 - iter 7/11 - loss 4.34334321
2019-11-09 19:43:58,317 epoch 8 - iter 8/11 - loss 4.23089108
2019-11-09 19:43:58,846 epoch 8 - iter 9/11 - loss 4.48738385
2019-11-09 19:43:59,315 epoch 8 - iter 10/11 - loss 4.40320122
2019-11-09 19:43:59,985 ----------------------------------------------------------------------------------------------------
2019-11-09 19:43:59,987 EPOCH 8 done: loss 4.4032 - lr 0.0500 - bad

{'test_score': 0.0,
 'dev_score_history': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0227,
  0.0227,
  0.0227,
  0.0227,
  0.0227],
 'train_loss_history': [4.897037376057018,
  4.741062727841464,
  4.584081519733775,
  4.761658560145985,
  4.458690730008212,
  4.458395437760786,
  4.454404917630282,
  4.403201222419739,
  4.352451627904719,
  4.26467193256725],
 'dev_loss_history': [tensor(6.9652, device='cuda:0'),
  tensor(6.7992, device='cuda:0'),
  tensor(7.4802, device='cuda:0'),
  tensor(6.4062, device='cuda:0'),
  tensor(6.9347, device='cuda:0'),
  tensor(6.2401, device='cuda:0'),
  tensor(6.4082, device='cuda:0'),
  tensor(6.3178, device='cuda:0'),
  tensor(6.2472, device='cuda:0'),
  tensor(6.2076, device='cuda:0')]}

In [24]:
# 8. plot weight traces (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_weights('resources/taggers/ag_news/weights.txt')

In [26]:
classifier = TextClassifier.load('resources/taggers/ag_news/final-model.pt')

# create example sentence
sentence = Sentence('France is the current world cup winner.')

# predict class and print
classifier.predict(sentence)

print(sentence.labels)

2019-11-09 19:45:24,662 loading file resources/taggers/example-ner/final-model.pt
I love Berlin


## Load/Use IMDB-Trained Sentiment Classifier Model

In [1]:
from flair.data import Sentence
from flair.models import TextClassifier

classifier = TextClassifier.load('/home/prw0007/.flair/models/imdb.pt')

# create example sentence
sentence = Sentence('France is the current world cup winner.')

# predict class and print
classifier.predict(sentence)

print(sentence.labels)

2019-11-09 19:59:49,339 loading file /home/prw0007/.flair/models/imdb.pt


/share/apps/rc/software/Anaconda3/5.3.1/envs/DeepNLP/lib/python3.6/site-packages/torch/serialization.py:573: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


[POSITIVE (0.9321351051330566)]
